In [2]:
import pandas as pd
import numpy as np
import string
import re

import nltk
from nltk.tokenize import word_tokenize

import os
from datetime import datetime
import json


# Ensure you have the necessary NLTK tokenizer models downloaded
# nltk.download('punkt')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
# Load an Excel file into a DataFrame
df = pd.read_excel('LCReviewsIntegrated_1962-1994.xlsx', engine='openpyxl')

In [ ]:
def remove_punctuation(input_string):
    # Create a translation table that maps each punctuation character to None
    translator = str.maketrans('', '', string.punctuation)
    # Translate the input string using the translation table
    return input_string.translate(translator)

In [18]:
def remove_extra_spaces(text):
    if not (type(text) == str):
        return np.nan
    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text.strip()

In [19]:
df['content'] = df['content'].apply(remove_extra_spaces)
df['title1'] = df['title1'].apply(remove_extra_spaces)
df['title2'] = df['title2'].apply(remove_extra_spaces)

In [22]:
title_in_content = []
title3 = []
for index, row in df.iterrows():
    if (type(row['title2']) == str):
        if row['title2'] in row['content']:
            title_in_content.append(1)
            title3.append(row['title2'])
        else:
            title_in_content.append(0)
            title3.append(row['title1'])
    else:
        title_in_content.append(0)
        title3.append(row['title1'])

In [23]:
sum(title_in_content) / len(title_in_content)

0.34390902081727065

In [24]:
df['title3'] = title3
df['title_in_content'] = title_in_content

In [25]:
df.to_excel('to_clean.xlsx', index=False, engine='openpyxl')

In [26]:
title_in_content2 = []
title4 = []
for index, row in df.iterrows():
    if (type(row['title2']) == str):
        if row['title2'] in row['content']:
            title_in_content2.append(1)
            title4.append(row['title2'])
        elif row['title2'].split("/")[0] in row['content']:
            title_in_content2.append(1)
            title4.append(row['title2'])
        else:
            title_in_content2.append(0)
            title4.append(row['title1'])
    else:
        title_in_content2.append(0)
        title4.append(row['title1'])

In [27]:
sum(title_in_content2) / len(title_in_content2)

0.3446414803392444

In [8]:
def extract_title(x):
    if not (type(x.title1) == str):
        return np.nan
    if not (type(x.content) == str):
        return np.nan
    if x.title1.lower() in x.content.lower():
        return x.title1

    sentence_parts = re.split(r' / | : ', x.title1.lower())
    sentence_parts = sorted(sentence_parts, key=len, reverse=True)
    for part in sentence_parts:
        if part in x.content.lower():
            return part
        elif remove_punctuation(part) in x.content.lower():
            return remove_punctuation(part)

    if remove_punctuation(x.title1).lower() in x.content.lower():
        return remove_punctuation(x.title1)

    return "error"

In [9]:
def find_sentence_in_text(full_text, sentence):
    start_index = full_text.find(sentence)
    if start_index == -1:
        print("EROR!!!")
        return False
    end_index = start_index + len(sentence)
    return start_index, end_index

In [10]:
result = []
for index, row in df.iterrows():
    result.append(extract_title(row))

df['title3'] = result

In [11]:
df_clean = df[df['title3'] != 'error']